#  Mini-batch learning a hard function with SDProp+

SDProp is a mini-batch learning algorithm very similar to RMSProp. RMSProp divides the loss function gradient by the square root of a running average of the gradient squared, while SDProp divides by the square root of a running average of the gradient's variance. In fact, SDProp is called centered RMSProp in pytorch. More on SDProp: https://www.ijcai.org/proceedings/2017/0267.pdf.

SDProp can, however be viewed as multiplying the gradients by the inverse square root of a running average of the gradient's covariance matrix. Where one makes the approximation of a diagonal covariance matrix. This point of view, invites us (at least me) to make other approximations of the covariance matrix. This is what I tried here.

## Experiment

A neural network with three hidden layers of size 16 with tanh activations was trained with three different optimizers to try and copy 2000 samples of a one-dimensional oscillating function ( $sin(50*(x+0.5))+((x+0.5)/0.5)^2 $) for x values between -0.5 and 0.5.

The three optimizers were:
- SDProp (Diagonal covariance matrix aka. centered RMSProp beta=0.99)
- Adam (Standard  beta parameters)
- SDProp (Model full covariance matrix within each layer. beta=0.99). $C^{-1/2}$ was calculated at each iteration using svd.

Batch size was 100 and number of iterations was 4000. Learning rate was 1e-3 for all optimizers.

Time usage:
- SDProp (Diagonal cov-matrix): 6.3 sec
- Adam : 2.8 sec. (Faster than SDProp likely since my python implementation of sdprop has a if-else in inner loop)
- SDProp (Layerwise covariances): 99 sec. 

### Learning curve
<img src="learning_curve.png">

### Plot
<img src="Figure_1.png">

## Discussion
Modeling covariances of the loss gradient made the learning converge for this specific example. Even though the full covariance matrix was not used, and only covariances within each layer of the neural net was modeled, the algorithm is still slow since the SVD's of the gradient's covariance matrix for each layer has to be computed at each iteration. But maybe there are cases where this computational cost is not prohibitive (Deep nets with small layers , RNNs, CNNs) where such an approach could give gains. More experiments are needed to figure this out. And my laptop is not up for the task, so feel free to see if you can find such a case. 

Thanks for reading.

